In [1]:
import torch

# 다중 선형 회귀
- 단순 선형 회귀와 달리 x의 값이 여러개이다.
- 3개인 경우 : H(x) = w₁x₁ + w₂x₂ + w₃x₃ + b

# PyTorch로 구현

![](../img/MLR.png)

In [2]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
torch.manual_seed(1)

In [4]:
x1_train = torch.FloatTensor([[73], [93], [89], [96], [73]])
x2_train = torch.FloatTensor([[80], [88], [91], [98], [66]])
x3_train = torch.FloatTensor([[75], [93], [90], [100], [70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [5]:
# 가중치 w와 편향 b 초기화
w1 = torch.zeros(1, requires_grad=True)
w2 = torch.zeros(1, requires_grad=True)
w3 = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [6]:
# optimizer 설정
optimizer = optim.SGD([w1, w2, w3, b], lr=1e-5)

In [7]:
nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = x1_train * w1 + x2_train * w2 + x3_train * w3 + b

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} w1: {:.3f} w2: {:.3f} w3: {:.3f} b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, w1.item(), w2.item(), w3.item(), b.item(), cost.item()
        ))

Epoch    0/1000 w1: 0.294 w2: 0.294 w3: 0.297 b: 0.003 Cost: 29661.800781
Epoch  100/1000 w1: 0.674 w2: 0.661 w3: 0.676 b: 0.008 Cost: 1.563634
Epoch  200/1000 w1: 0.679 w2: 0.655 w3: 0.677 b: 0.008 Cost: 1.497607
Epoch  300/1000 w1: 0.684 w2: 0.649 w3: 0.677 b: 0.008 Cost: 1.435026
Epoch  400/1000 w1: 0.689 w2: 0.643 w3: 0.678 b: 0.008 Cost: 1.375730
Epoch  500/1000 w1: 0.694 w2: 0.638 w3: 0.678 b: 0.009 Cost: 1.319511
Epoch  600/1000 w1: 0.699 w2: 0.633 w3: 0.679 b: 0.009 Cost: 1.266222
Epoch  700/1000 w1: 0.704 w2: 0.627 w3: 0.679 b: 0.009 Cost: 1.215696
Epoch  800/1000 w1: 0.709 w2: 0.622 w3: 0.679 b: 0.009 Cost: 1.167818
Epoch  900/1000 w1: 0.713 w2: 0.617 w3: 0.680 b: 0.009 Cost: 1.122429
Epoch 1000/1000 w1: 0.718 w2: 0.613 w3: 0.680 b: 0.009 Cost: 1.079378


# 벡터와 행렬 연산

- x의 개수가 많아질수록 위의 방법처럼 일일이 선언하기 힘들다.
- 행렬 곱셈(또는 벡터의 내적)을 이용하여 문제를 해결한다.
- 벡터의 내적 : 행렬 곱셈 과정에서 이루어지는 벡터 연산
![행렬곱셈 중 벡터의 내적 연산 과정](../img/MLR1.png)
- 1*7 + 2*9 + 3*11 = 58

## 벡터 연산으로 표현

> H(X) = w₁x₁ + w₂x₂ + w₃x₃
### ![벡터의 내적](../img/MLR2.png)
> H(X) = XW (두 벡터를 각각 X와 W로 표현할 경우)

## 행렬 연산으로 표현

### ![](../img/MLR.png)
- sample : 전체 훈련 데이터 개수를 셀 수 있는 단위 (예시 샘플의 개수 : 5개)
- feature(특성) : 각 샘플의 y를 결정하는 독립변수 x (예시 특성의 개수 : 3개)
- 독립변수 x의 개수 : sample * feature (5 * 3 = 15) => 행렬 X
### ![행렬 X](../img/MLR3.png)
- 벡터 W를 곱한다.
### ![H(X) = XW](../img/MLR4.png)
> H(X) = XW
- 편향 b를 추가한다.
### ![H(X) = XW + B](../img/MLR5.png)
> H(X) = XW + B

# 행렬 연산으로 PyTorch 구현

In [9]:
x_train  =  torch.FloatTensor([[73,  80,  75], 
                               [93,  88,  93], 
                               [89,  91,  80], 
                               [96,  98,  100],   
                               [73,  66,  70]]) # 5 * 3
y_train  =  torch.FloatTensor([[152],  [185],  [180],  [196],  [142]]) # 5 * 1

In [8]:
# 가중치와 편향 선언
W = torch.zeros((3, 1), requires_grad=True) # 3 * 1
b = torch.zeros(1, requires_grad=True)

In [10]:
hypothesis = x_train.matmul(W) + b # H(X) = XW + B

In [13]:
# 완성
# 데이터 선언 생략

# optimizer 설정
optimizer = optim.SGD([W, b], lr=1e-5)

nb_epochs = 5000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    # 편향 b는 브로드 캐스팅되어 각 샘플에 더해집니다.
    hypothesis = x_train.matmul(W) + b

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(
        epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()))

Epoch    0/5000 hypothesis: tensor([153.8917, 184.8801, 176.6712, 198.0892, 141.2973]) Cost: 3.906476
Epoch  100/5000 hypothesis: tensor([153.8744, 184.8621, 176.7496, 198.0428, 141.3073]) Cost: 3.750101
Epoch  200/5000 hypothesis: tensor([153.8570, 184.8449, 176.8258, 197.9976, 141.3176]) Cost: 3.600789
Epoch  300/5000 hypothesis: tensor([153.8396, 184.8285, 176.8999, 197.9534, 141.3281]) Cost: 3.458147
Epoch  400/5000 hypothesis: tensor([153.8220, 184.8129, 176.9720, 197.9101, 141.3388]) Cost: 3.321836
Epoch  500/5000 hypothesis: tensor([153.8045, 184.7981, 177.0420, 197.8678, 141.3497]) Cost: 3.191587
Epoch  600/5000 hypothesis: tensor([153.7869, 184.7839, 177.1102, 197.8264, 141.3609]) Cost: 3.067054
Epoch  700/5000 hypothesis: tensor([153.7693, 184.7705, 177.1764, 197.7860, 141.3722]) Cost: 2.947954
Epoch  800/5000 hypothesis: tensor([153.7517, 184.7577, 177.2408, 197.7465, 141.3836]) Cost: 2.834051
Epoch  900/5000 hypothesis: tensor([153.7341, 184.7455, 177.3035, 197.7078, 141.39